In [22]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import word_tokenize
import numpy as np
import math
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DOLPHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
#...........import Bangla stopword................
df = pd.read_excel('bng_sw.xlsx', sheet_name=0)
bng_stopword = df['Stopwords'].tolist()

In [24]:
#............read article.................
f = open("test.txt", encoding = 'utf-8')
text = f.read()
# print("Test Data Is: ",text)

# .........split every sentence & store an array............
sentence1 = text.split("।")
np_sent1 = np.array(sentence1)

# .........remove last row for blank line...........
np_sent1 = np.delete(np_sent1,-1)

#.........Store original sentences in an array.......
org_sent = []
for sentence in np_sent1:
    org_sent.append(sentence)
print(org_sent)

['\ufeffভালো-মন্দের মধ্য দিয়েই ২০১৯ সাল পার করল বাংলাদেশ', ' সারা বছর ধরেই নানা আন্তর্জাতিক সংস্থা বিভিন্ন সূচক প্রকাশ করে থাকে', ' সার্বিকভাবে এসব সূচক দিয়ে বহির্বিশ্বে বাংলাদেশের অবস্থান নির্ধারণ করা হয়', ' তবে আশঙ্কার বিষয় হলো কিছু সূচকে এত দিন ইতিবাচক ধারায় থাকলেও এখন চ্যালেঞ্জের মুখে পড়ছে', ' ১৬০ দেশের মধ্যে বাংলাদেশ ১৩৪তম']


In [25]:
def remove_stopword(sentence_array):
    for i in range(len(sentence_array)):
        text_tokens = word_tokenize(sentence_array[i])
        tokens_without_sw = [word for word in text_tokens if not word in bng_stopword]
        filtered_text = (" ").join(tokens_without_sw)
        sentence_array[i] = filtered_text

#     print("After Remove Stop-Word:\n", np_sent1)
    return(sentence_array)
    

In [26]:
def count_actual_token(sentence_array):
    actual_tokens = []
    for i in range(len(sentence_array)):
        text_tokens = word_tokenize(sentence_array[i])
        for j in range(len(text_tokens)):
            actual_tokens.append(text_tokens[j])

    actual_tokens = list(dict.fromkeys(actual_tokens))
    return actual_tokens

In [27]:
def countX(lst, x): 
    count = 0
    for ele in lst: 
        if (ele == x): 
            count = count + 1
    return count 

In [28]:
def count_TF_score(sent_array,token):
    rows, cols = (len(sent_array), len(token)) 
    tf_score = [[0 for i in range(cols)] for j in range(rows)]

    for i in range(rows):
        text_tokens = word_tokenize(sent_array[i])
        count = 0
        total_term = len(text_tokens)
        for j in range(cols):
            rep_term = countX(text_tokens,token[j])
            count = (rep_term/total_term);
            formatted_float = "{:.5f}".format(count)
            tf_score[i][j] = formatted_float
            
    return tf_score

In [29]:
def count_IDF_score(sent_array,token):
    idf_score = []

    for i in range(len(token)):
        count_idf = 0
        score = 0
        for j in range(len(sent_array)):
            sentence = word_tokenize(sent_array[j])
            term = token[i]
            for ele in sentence:
                if (ele == term):
                    count_idf = count_idf + 1
                    break
        score = math.log(len(sent_array)/count_idf,10)
        formatted_float = "{:.5f}".format(score)
        idf_score.append(formatted_float)
    
    return idf_score

In [30]:
def count_TF_IDF_score(np_sentences,token_list):
    rows, cols = (len(np_sentences), len(token_list)) 
    tf_idf_score = [[0 for i in range(cols)] for j in range(rows)]
    
    #........Count TF score of each sentence..............
    tf_score = count_TF_score(np_sentences,token_list)
    
    #........Count IDF score of each sentence..............
    idf_score = count_IDF_score(np_sentences,token_list)

    for i in range (rows):
        for j in range(cols):
            tf = tf_score[i][j]
            idf = idf_score[j]
            score = float(tf) * float(idf)
            formatted_float = "{:.5f}".format(score)
            tf_idf_score[i][j] = formatted_float
           
    return tf_idf_score

In [31]:
#.........Remove Stop-Word From Each Sentence........
filtered_sentence = remove_stopword(np_sent1)
# print(filtered_sentence)

#........Count actual token of full article..........
actual_tokens = count_actual_token(filtered_sentence)
# print(actual_tokens)

#........Count TF-IDF score of each sentence..........
tf_idf_vector = count_TF_IDF_score(filtered_sentence,actual_tokens)

In [32]:
#.......Print Output.........
for i in range(len(org_sent)):
    print("Sentence: ", org_sent[i])
    print("Vector: ", tf_idf_vector[i][:])

Sentence:  ﻿ভালো-মন্দের মধ্য দিয়েই ২০১৯ সাল পার করল বাংলাদেশ
Vector:  ['0.08737', '0.08737', '0.08737', '0.08737', '0.08737', '0.08737', '0.08737', '0.04974', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000']
Sentence:   সারা বছর ধরেই নানা আন্তর্জাতিক সংস্থা বিভিন্ন সূচক প্রকাশ করে থাকে
Vector:  ['0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.09985', '0.09985', '0.09985', '0.09985', '0.09985', '0.05685', '0.09985', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000']
Sentence:   সার্বিকভাবে এসব সূচক দিয়ে বহির্বিশ্বে বাংলাদেশের অবস্থান নির্ধারণ করা হয়
Vector: